In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split


In [2]:
# Import Data file from Data Wrangling Process

df = pd.read_csv('Image Values2.csv')

df.head()

,Unnamed: 0,Image Name,Image Value
0,0,A1,A
1,1,A2,A
2,2,A3,A
3,3,A4,A
4,4,A5,A


# Apply pandas.get_dummies to convert categorical data to numerical

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import numpy as np

In [7]:
alphabet = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']
train = df
train_image = []
for letter in alphabet:
    for i in tqdm([*range(1,3001)]):
        img = f'C:/Bootcamp/Homework/proj3-team04/asl_alphabet_train/asl_alphabet_train/{letter}/{letter}{i}.jpg'
        img = image.load_img(img,target_size=(28,28,3),color_mode = "rgb")
        img = image.img_to_array(img)
        img = img/255
        train_image.append(img)
    

100%|█████████████████████████████████████████████████████████████████████████████| 3000/3000 [00:23<00:00, 127.99it/s]


In [8]:
# Convert image arrays to numpy array

X = np.array(train_image)



In [9]:
y=train['Image Value'].values
#y = pd.get_dummies(y)


In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

In [11]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)


In [12]:
y_train_cat = to_categorical(encoded_y_train)
y_test_cat = to_categorical(encoded_y_test)

In [13]:
y_train.shape


(62400,)

In [14]:
X_train.shape

(62400, 28, 28, 3)

In [15]:
model = Sequential()
model.add(Conv2D(26, kernel_size=(3, 3),activation='relu',input_shape=(28, 28, 3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
#model.add(Dropout(0.5))
model.add(Dense(26, activation='softmax'))

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 26)        728       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        15040     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 9216)              0         
_________________________________________________________________
dense (Dense)                (None, 64)                589888    
_________________________________________________________________
dense_1 (Dense)              (None, 26)                1690      
Total params: 607,346
Trainable params: 607,346
Non-trainable params: 0
__________________________________________________

In [17]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [18]:
model.fit(X_train, y_train_cat, epochs=10, validation_data=(X_test, y_test_cat))

Epoch 1/10
1950/1950 [==============================] - 111s 57ms/step - loss: 0.9755 - accuracy: 0.6972 - val_loss: 0.3231 - val_accuracy: 0.8938
Epoch 2/10
1950/1950 [==============================] - 109s 56ms/step - loss: 0.2034 - accuracy: 0.9329 - val_loss: 0.1698 - val_accuracy: 0.9431
Epoch 3/10
1950/1950 [==============================] - 112s 58ms/step - loss: 0.1061 - accuracy: 0.9647 - val_loss: 0.1353 - val_accuracy: 0.9526
Epoch 4/10
1950/1950 [==============================] - 113s 58ms/step - loss: 0.0721 - accuracy: 0.9763 - val_loss: 0.1089 - val_accuracy: 0.9607
Epoch 5/10
1950/1950 [==============================] - 111s 57ms/step - loss: 0.0558 - accuracy: 0.9816 - val_loss: 0.0739 - val_accuracy: 0.9756
Epoch 6/10
1950/1950 [==============================] - 109s 56ms/step - loss: 0.0461 - accuracy: 0.9852 - val_loss: 0.0488 - val_accuracy: 0.9833
Epoch 7/10
1950/1950 [==============================] - 104s 53ms/step - loss: 0.0394 - accuracy: 0.9878 - val_loss: 0

In [19]:
encoded_predictions = model.predict_classes(X_test[:20])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [20]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels    : {(y_test[:20])}")

Predicted classes: ['U' 'O' 'I' 'M' 'G' 'J' 'C' 'K' 'S' 'C' 'V' 'D' 'B' 'K' 'K' 'P' 'V' 'J'
 'Q' 'N']
Actual Labels    : ['U' 'O' 'I' 'M' 'G' 'J' 'C' 'K' 'S' 'C' 'V' 'D' 'B' 'K' 'K' 'P' 'V' 'J'
 'Q' 'N']
